# WEEKLY ANALYSIS FOR CARD TRENDS!

In [111]:
import json
from collections import Counter
import plotly.express as px
import pandas as pd

In [75]:
with open("opponent_decks.json", "r") as f:
    data = json.load(f)

In [76]:
print(data)

{'20251121T175245.000Z': {'Knight': {'level': 11, 'elixir': 3, 'rarity': 'common'}, 'Baby Dragon': {'level': 6, 'elixir': 4, 'rarity': 'epic'}, 'Ice Wizard': {'level': 3, 'elixir': 3, 'rarity': 'legendary'}, 'Tombstone': {'level': 10, 'elixir': 3, 'rarity': 'rare'}, 'Poison': {'level': 6, 'elixir': 4, 'rarity': 'epic'}, 'Graveyard': {'level': 4, 'elixir': 5, 'rarity': 'legendary'}, 'Tornado': {'level': 6, 'elixir': 3, 'rarity': 'epic'}, 'Barbarian Barrel': {'level': 5, 'elixir': 2, 'rarity': 'epic'}}, '20251121T174848.000Z': {'Bandit': {'level': 3, 'elixir': 3, 'rarity': 'legendary'}, 'Bats': {'level': 11, 'elixir': 2, 'rarity': 'common'}, 'Royal Ghost': {'level': 3, 'elixir': 3, 'rarity': 'legendary'}, 'Ram Rider': {'level': 3, 'elixir': 5, 'rarity': 'legendary'}, 'Mega Knight': {'level': 3, 'elixir': 7, 'rarity': 'legendary'}, 'Archer Queen': {'level': 1, 'elixir': 5, 'rarity': 'champion'}, 'Arrows': {'level': 10, 'elixir': 3, 'rarity': 'common'}, 'Lightning': {'level': 6, 'elixir': 

Genereert een hele grote lijst met dicts waarin elke kaart een timestamp krijgt

In [77]:
wrangled = []
current_date = None
for d in data:
    if current_date is None or d:
        current_date = d
    current_date = current_date[0:4] + '-' + current_date[4:6] + '-' + current_date[6:8] 
    for c in range(len(d)):
        card_data = data[d]
        cards = card_data.keys()
        for card in cards:
            wrangled.append({"timestamp": current_date, "card": card})

df = pd.DataFrame(wrangled)
print(df)

        timestamp             card
0      2025-11-21           Knight
1      2025-11-21      Baby Dragon
2      2025-11-21       Ice Wizard
3      2025-11-21        Tombstone
4      2025-11-21           Poison
...           ...              ...
26875  2025-11-29     Mother Witch
26876  2025-11-29  Skeleton Barrel
26877  2025-11-29          Furnace
26878  2025-11-29      Cannon Cart
26879  2025-11-29    Skeleton Army

[26880 rows x 2 columns]


nu voegen we een kolom toe aan de dataframe

In [78]:
df["timestamp"] = pd.to_datetime(df["timestamp"])
df['week'] = df['timestamp'].dt.to_period("W")

print(df)

       timestamp             card                   week
0     2025-11-21           Knight  2025-11-17/2025-11-23
1     2025-11-21      Baby Dragon  2025-11-17/2025-11-23
2     2025-11-21       Ice Wizard  2025-11-17/2025-11-23
3     2025-11-21        Tombstone  2025-11-17/2025-11-23
4     2025-11-21           Poison  2025-11-17/2025-11-23
...          ...              ...                    ...
26875 2025-11-29     Mother Witch  2025-11-24/2025-11-30
26876 2025-11-29  Skeleton Barrel  2025-11-24/2025-11-30
26877 2025-11-29          Furnace  2025-11-24/2025-11-30
26878 2025-11-29      Cannon Cart  2025-11-24/2025-11-30
26879 2025-11-29    Skeleton Army  2025-11-24/2025-11-30

[26880 rows x 3 columns]


Nu tellen we de kaarten per periode

ik snap niet hoe het precies werkt maar het groepeert dus de kaarten op week en kaart en telt dan over die groepen hoeveel ze worden gespeeld

In [85]:
counted_df = df.groupby(['week','card']).aggregate('count')
print(counted_df)

                                    timestamp
week                  card                   
2025-11-17/2025-11-23 Archer Queen         40
                      Archers              60
                      Arrows              240
                      Baby Dragon          60
                      Balloon              40
...                                       ...
2025-11-24/2025-11-30 Witch               520
                      Wizard              260
                      X-Bow                80
                      Zap                 560
                      Zappies             100

[197 rows x 1 columns]


Nu nog reindexen en flippen!

In [165]:
idx_counted_df = counted_df.reset_index()


#kleine rename voorduidelijkheid
idx_counted_df['count'] = idx_counted_df.pop('timestamp')

#de grote wisseltruc
pivot_df = idx_counted_df.pivot(index='week',columns='card',values='count')
# #beetje opruimen
clean_df = pivot_df.fillna(0)

clean_df = clean_df.reset_index()

print(clean_df)

card                   week  Archer Queen  Archers  Arrows  Baby Dragon  \
0     2025-11-17/2025-11-23          40.0     60.0   240.0         60.0   
1     2025-11-24/2025-11-30         180.0    340.0   640.0        360.0   

card  Balloon  Bandit  Barbarian Barrel  Barbarians   Bats  ...  Tornado  \
0        40.0    60.0              60.0         0.0   60.0  ...     40.0   
1       200.0   200.0             400.0        80.0  440.0  ...    220.0   

card  Valkyrie  Vines  Void  Wall Breakers  Witch  Wizard  X-Bow    Zap  \
0         80.0   20.0   0.0            0.0  100.0    20.0    0.0   40.0   
1        340.0  120.0  20.0          160.0  520.0   260.0   80.0  560.0   

card  Zappies  
0        20.0  
1       100.0  

[2 rows x 114 columns]


# PLOT!!!

In [ ]:
#for week in idx_counted_df['week']:
clean_df['week'] = clean_df['week'].astype(str)


print(clean_df['week'].dtypes)
print('')
print(clean_df[clean_df.columns[0]].head())


object

0    2025-11-17/2025-11-23
1    2025-11-24/2025-11-30
Name: week, dtype: object


# NU WEL PLOT WANT DATASET WAS SCHIJT

In [168]:

fig = px.line(
    clean_df,
    x = clean_df.columns[0],
    y = clean_df.columns[1:],
    title= "weekly trend of"
)
fig.show()

In [ ]:
#TODO ZET OM NAAR PERCENTAGES EN VERANDER PERIODES!!